In this project, we'll put together a mobile app designed to help lottery addicts realistically understand their chances of winning.

Many people start playing the lottery for fun, but for some, this activity can escalate into habit and addiction. Similar to other compulsive gamblers, lottery addicts may start using savings and loans to buy tickets. They start to accumulate debts and eventually resort to more desperate behaviors like theft, deceiving to family, or worse.

We are an data analyst at a medical institute. We are assigned to assist in the development of a mobile app intended to guide lottery addicts through exercises that will let them better estimate their chances of winning. The hope is that this app will help them realize that buying too many tickets will do little to improve their chances of winning. The institute has a team of engineers that will build the app, but they need us to build the logic behind the app and calculate probabilities.

For the first version of the app, they want us to focus on the [6/49](https://en.wikipedia.org/wiki/Lotto_6/49) lottery and build functions that can answer users questions like:

* What is the probability of winning the big prize with a single ticket?
* What is the probability of winning the big prize if we play 40 different tickets (or any other number)?
* What is the probability of having at least five (or four, or three, or two) winning numbers on a single ticket?

The institute also wants us to consider historical data coming from the national 6/49 lottery game in Canada. [The data set](https://www.kaggle.com/datascienceai/lottery-dataset) has data for 3,665 drawings, dating from 1982 to 2018. The 6/49 lottery is one of the first Canadian lotteries game to allow players to pick their own numbers. We can download the actual data set [here](https://dsserver-prod-resources-1.s3.amazonaws.com/409/649.csv) to have a copy on our local machine.

[solution](https://github.com/dataquestio/solutions/blob/master/Mission409Solutions.Rmd)

#### Developing A Mobile App For Alleviating Lottery Addiction

This RMarkdown file is intended to lay out the logic of a mobile app designed for those addicted to the lottery. By showing a user how to calculate the incredibly small probabilities of winning the lottery, we hope that the app will help them better grasp that buying multiple lottery tickets will do little to help them win. Through this understanding, they will hopefully stop purchasing lottery tickets in an unhealthy manner.

We laid out our goal of writing out the probability logic for a lottery mobile app. Throughout the project, we'll need to calculate repeatedly probabilities and combinations, so wrapping it in a function will save us a lot of time. We'll start by writing two functions:

1. A function that calculates factorials and
2. A function that calculates the numbers of combinations.

In the 6/49 lottery, six numbers are drawn from a set of 49 numbers that range from 1 to 49. The drawing is done without replacement, so once a number is drawn, it's not put back in the set.

### Core Functions

```{r}
`factorial <- function(n) {
  product = 1
  for (i in 1:n) {
    product = product * i
  }
  return(product)
}`

`combinations <- function(n, k) {
  numerator <- factorial(n)
  denominator <- factorial(k) * factorial(n - k)
  return(numerator / denominator)
}`


We use build upon these functions and create another function to calculate the probability of winning the big prize.

In the 6/49 lottery, six numbers are drawn from a set of 49 numbers that range from 1 to 49. A player wins the big prize if the six numbers on their tickets match all the six numbers drawn. If a player has a ticket with the numbers {13, 22, 24, 27, 42, 44}, he only wins the big prize if the numbers drawn are {13, 22, 24, 27, 42, 44}. Even if just one number differs, they won't win.

For the first version of the app, we want players to be able to calculate the probability of winning the big prize with the various numbers they play on a single ticket (for each ticket a player chooses six numbers out of 49). So, we'll start by building a function that calculates the probability of winning the big prize for any given ticket.

We discussed with the engineering team of the medical institute, and they told us we need to be aware of the following details when we write the function:

* Inside the app, the user inputs six different numbers from 1 to 49.
* Under the hood, the six numbers will come as an R vector, which will serve as the single input to our function.
* The engineering team wants the function to print the probability value in a friendly way — in a way that people without any probability training are able to understand.

### One-Ticket Probability

```{r}
`one_ticket_probability <- function(nums) {
  total_combinations <- combinations(49, 6)
  prob <- (1 / total_combinations) * 100
  pretty_prob <- sprintf("%1.9f", prob)
  s <- paste("You have a ", pretty_prob, "% chance of winning the big prize.", sep = "")
  return(s)
}`

`one_ticket_probability(c(1, 2, 3, 4, 5, 6))`

We wrote a function that can tell users what is the probability of winning the big prize with a single ticket. For the first version of the app, however, users should also be able to compare their ticket against past winning combinations in the historical lottery data in Canada. Having this functionality will allow users to determine whether they would have ever won by now.

We'll focus on exploring the historical data coming from the Canada 6/49 lottery. 

The data set contains historical data for 3,665 drawings (each row shows data for a single drawing), dating from 1982 to 2018. For each drawing, we can find the six numbers drawn in the following six columns:

* `NUMBER DRAWN 1`
* `NUMBER DRAWN 2`
* `NUMBER DRAWN 3`
* `NUMBER DRAWN 4`
* `NUMBER DRAWN 5`
* `NUMBER DRAWN 6`

### Historical Data Check for Canada Lottery

```{r, message = FALSE, warning = FALSE}

`library(tidyverse)
lottery649 <- read_csv("649.csv")
print(dim(lottery649))`



`head(lottery649, 3)`

`tail(lottery649, 3)`


Before we perform start working more with the historical data, we'll take a quick aside to learn an important function for the project. Before we move on and analyze it, it's worthwhile to take some time to learn some new R features. We learned about the different `map` functions. The `map` functions are great for when we want to apply the same function over many different values. As we currently know the function, we only know how to iterate over a single set of values.

In the `649` lottery dataset, each of the drawn numbers are actually in different columns! If we try to use the `map` function, we only know how to put in one column at a time. We'll eventually want to take in multiple columns simultaneously, so we'll need to learn an extension of the `map` function to handle this.

We'll learn the `pmap` function. While `map` allows us to iterate over just one vector with a function, `pmap` allow us to use multiple vectors simultaneously in the function.

Like `map`, `pmap` takes two inputs: the first is some data, and the second is a function that we want to use to process the data. The crucial difference between the two functions is that the data given to `pmap` must be in a particular structure. In `map`, we only need to pass in a vector and function. In `pmap`, the data needs to be passed in as a list. A list is an extremely flexible data structure available in R and can be thought of as a vector that can contain other vectors.

The order of the vectors in a list matters a lot when we use anonymous functions in `pmap`. The first vector in the list will act as the first input in the anonymous function. The second will act as the second vector, and so on. We need to keep this order in mind when we're coding the anonymous function.

`data1 <- c(1, 3, 5)
data2 <- c(2, 4, 6)
data3 <- c(8, 9, 7)`

`data_list <- list(data1, data2, data3)`

`sums <- pmap(data_list, function(x, y, z) { x + y + z })`

`pmap` starts by taking the first item in each of the vectors of the list, `data1`, `data2`, `data3`. These first items then become the inputs to the anonymous function. The first item in `data1` is used as the `x` input, the first item from `data2` is used as `y`, and finally the first item from `data3` is used as `z`. Then, these items are summed up, as described in the anonymous function. After the sum is calculated, it is stored and then the process is repeated on the second items from each vector, and so on.

`pmap` does not return a single vector, but rather another list! Since the output is also a list.  If we'd like to turn a list back into an array, R has a convenient function called [`unlist()`](https://stat.ethz.ch/R-manual/R-devel/library/base/html/unlist.html). Given a list, `unlist` will convert it back into a vector.

We're going to write a function that will enable users to compare their ticket against the historical lottery data in Canada and determine whether they would have ever won by now.

The engineering team told us that we need to be aware of the following details:

* Inside the app, the user inputs six different numbers from 1 to 49.
* Under the hood, the six numbers will come as an R vector and serve as an input to our function.
* The engineering team wants us to write a function that prints:
    * the number of times the combination selected occurred in the Canada data set and
    * the probability of winning the big prize in the next drawing with that combination.

### Function for Historical Data Check


```{r}

`istorical_lots <- pmap(
  list(
    u <- lottery649$`NUMBER DRAWN 1`,
    v <- lottery649$`NUMBER DRAWN 2`,
    w <- lottery649$`NUMBER DRAWN 3`,
    x <- lottery649$`NUMBER DRAWN 4`,
    y <- lottery649$`NUMBER DRAWN 5`,
    z <- lottery649$`NUMBER DRAWN 6`
  ), 
  f <- function(u, v, w, x, y, z) { c(u, v, w, x, y, z) }
  )`

```{r}

`library(sets)
check_historical_occurrences <- function(lot, hist_lots = historical_lots) {
  historical_matches <- map(hist_lots, function(x) {setequal(x, lot)})
  num_past_matches <- sum(unlist(historical_matches))
  s <- paste("The combination you entered has appeared ", 
             num_past_matches, 
             " times in the past. ",
             "Your chance of winning the big prize in the next drawing using this combination is 0.0000072%", sep = "")
  return(s)
}` # The [setequal()](https://stat.ethz.ch/R-manual/R-devel/library/base/html/sets.html) function  above come in handy here

`check_historical_occurrences(c(3, 12, 11, 14, 41, 43))
check_historical_occurrences(c(1, 2, 3, 4, 5, 6))`

So far, we've written two main functions for the app:

* `one_ticket_probability()` — calculates the probability of winning the big prize with a single ticket
* `check_historical_occurrence()` — checks whether a certain combination has occurred in the Canada lottery data set

One situation our functions do not cover is the issue of multiple tickets. Lottery addicts usually play more than one ticket on a single drawing, thinking that this might increase their chances of winning significantly. Our purpose is to help them better estimate their chances of winning.

We're going to write a function that will allow the users to calculate the chances of winning for any number of different tickets.

We've talked with the engineering team and they gave us the following information:

* The user will input the number of different tickets they want to play (without inputting the specific combinations they intend to play).
* Our function will see an integer between 1 and 13,983,816 (the maximum number of different tickets).
* The function should print information about the probability of winning the big prize depending on the number of different tickets played.

# Multi-ticket Probability

```{r}

`multi_ticket_probability <- function(n) {
  total_combinations <- combinations(49, 6)
  prob <- (n / total_combinations) * 100
  pretty_prob <- sprintf("%1.9f", prob)
  s <- paste("you have a ", pretty_prob, "% chance of winning the big prize.", sep = "")
  return(s)
}`


`test_amounts <- c(1, 10, 100, 10000, 1000000, 6991908, 13983816)`

`for (n in test_amounts) {
  print(paste("For ", n, " tickets, ",  multi_ticket_probability(n), sep = ""))
}`


So far, we wrote three main functions:

* `one_ticket_probability()` — calculates the probability of winning the big prize with a single ticket
* `check_historical_occurrence()` — checks whether a certain combination has occurred in the Canada lottery data set
* `multi_ticket_probability()` — calculates the probability for any number of of tickets between 1 and 13,983,816


Now we're going to write one more function to allow the users to calculate probabilities for three, four, or five winning numbers.

For extra context, in most 6/49 lotteries there are smaller prizes if a player's ticket matches three, four, or five of the six numbers drawn. As a consequence, the users might be interested in knowing the probability of having three, four, or five winning numbers.

These are the engineering details we'll need to be aware of:

* Inside the app, the user inputs:
    * six different numbers from 1 to 49; and
    * an integer between 3 and 5 that represents the number of winning numbers expected
* Our function prints information about the probability of having the inputted number of winning numbers.

To calculate the probabilities, we tell the engineering team that the specific combination on the ticket is irrelevant and we only need the integer between 3 and 5 representing the number of winning numbers expected. Consequently, we will write a function which takes in an integer and prints information about the chances of winning depending on the value of that integer.

### Less Winning Numbers

```{r}
`probability_less_6 <- function(n) {
    n_combinations_ticket = combinations(6, n)
    n_combinations_remaining = combinations(49 - n, 6 - n)
    successful_outcomes = n_combinations_ticket * n_combinations_remaining
    n_combinations_total = combinations(49, 6)
    prob = (successful_outcomes / n_combinations_total) * 100
    pretty_prob <- sprintf("%1.9f", prob)
    s <- paste("you have a ", pretty_prob, "% chance of winning the big prize.", sep = "")
    return(s)
}`


`winning_nums <- c(3, 4, 5)
for (n in winning_nums) {
  print(paste("For ", n, " tickets, ",  probability_less_6(n), sep = ""))
}`


That was all for the part of the project!

Possible features for a second version of the app include:

* Improve the `probability_less_6()` function to show the probabilities for having two winning numbers as well.
* Making the outputs even easier to understand by adding fun analogies (for example, we can find probabilities for strange events and compare with the chances of winning in lottery; for instance, we can output something along the lines "We are 100 times more likely to be the victim of a shark attack than winning the lottery").
* Combine the `one_ticket_probability()` and `check_historical_occurrence()` to output information on probability and historical occurrence at the same time.